In [ ]:
# ==============================================================================
# ENTERPRISE CREDIT SCORING SYSTEM v10.0
# ==============================================================================
# Autor: Sistema de Crédito Enterprise
# Compliance: BACEN Resolution 4.557/2017, LGPD, Basel III
# Última Atualização: 2025-10-28
# ==============================================================================

import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import json
import logging
import hashlib
import sys
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Any, Union
from dataclasses import dataclass, field, asdict
from abc import ABC, abstractmethod
from enum import Enum

# ML Libraries
import optuna
from scipy.stats import ks_2samp, chi2_contingency
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve,
    confusion_matrix, classification_report,
    brier_score_loss, log_loss, average_precision_score
)

# Explicabilidade
import shap

# Configuração Global
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler('credit_scoring.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# ==============================================================================
# 1. ENUMERADORES E CONSTANTES
# ==============================================================================

class ModelStatus(Enum):
    """Status do modelo no ciclo de vida."""
    DEVELOPMENT = "development"
    STAGING = "staging"
    PRODUCTION = "production"
    DEPRECATED = "deprecated"
    FAILED = "failed"

class AlertLevel(Enum):
    """Níveis de alerta para monitoramento."""
    INFO = "info"
    WARNING = "warning"
    CRITICAL = "critical"
    EMERGENCY = "emergency"

class DataQualityIssue(Enum):
    """Tipos de problemas de qualidade de dados."""
    MISSING_VALUES = "missing_values"
    OUTLIERS = "outliers"
    DATA_DRIFT = "data_drift"
    TARGET_LEAKAGE = "target_leakage"
    INVALID_VALUES = "invalid_values"

# Constantes de Negócio
BUSINESS_CONSTANTS = {
    'min_age': 18,
    'max_age': 85,
    'min_income': 0,
    'max_loan_amount': 1_000_000,
    'max_loan_term_months': 72,
    'excel_date_base': pd.Timestamp('1899-12-30'),  # Excel Windows base
    'excel_mac_date_base': pd.Timestamp('1904-01-01')
}

# Variáveis que indicam Target Leakage (nunca devem estar no modelo)
FORBIDDEN_FEATURES = [
    'status_financeiro', 'inadimplente', 'default',
    'quantidade_parcelas_vencidas', 'taxa_parcelas_vencidas',
    'saldo_vencido', 'dias_em_atraso', 'primeiro_vencimento_em_atraso',
    'data_quitacao', 'pagamento_efetuado', 'valor_pago',
    'parcelas_pagas', 'atraso_', 'vencido', 'quitado'
]

# ==============================================================================
# 2. CONFIGURAÇÃO CENTRALIZADA
# ==============================================================================

@dataclass
class BusinessMetrics:
    """Métricas de negócio para otimização."""
    revenue_per_good_client: float = 1000.0      # Receita média por bom pagador
    loss_per_bad_client: float = 5000.0          # Perda média por inadimplente
    operational_cost_per_analysis: float = 50.0   # Custo operacional por análise
    max_approval_rate: float = 0.70              # Taxa máxima de aprovação
    target_bad_rate: float = 0.05                # Taxa de inadimplência alvo (5%)
    discount_rate: float = 0.10                  # Taxa de desconto (10% a.a.)

@dataclass
class ModelThresholds:
    """Thresholds para alertas e ações."""
    min_auc_acceptable: float = 0.70
    min_ks_acceptable: float = 0.30
    max_psi_warning: float = 0.10
    max_psi_critical: float = 0.25
    max_feature_missing_rate: float = 0.30
    min_samples_per_class: int = 100
    max_class_imbalance_ratio: float = 20.0

@dataclass
class MLConfig:
    """Configuração de Machine Learning."""
    
    # Dados
    data_path: str = 'Base estatistica 14071.xlsx'
    sheet_name: str = 'cobranca-d-30'
    target_variable: str = 'status_financeiro'
    date_column: str = 'data_efetivacao'
    
    # Split Strategy
    test_size: float = 0.15
    validation_size: float = 0.15
    oot_months: int = 3  # Meses para Out-of-Time test
    
    # Cross-Validation
    n_folds_cv: int = 5
    cv_strategy: str = 'stratified'  # 'stratified' ou 'timeseries'
    
    # Otimização
    n_trials_optuna: int = 100
    optuna_timeout: int = 3600  # 1 hora
    early_stopping_rounds: int = 50
    
    # Modelos
    models_to_evaluate: List[str] = field(default_factory=lambda: [
        'xgboost', 'lightgbm', 'ensemble'
    ])
    
    # Feature Engineering
    create_polynomial_features: bool = True
    max_polynomial_degree: int = 2
    create_interaction_features: bool = True
    
    # Imputação
    numeric_imputation_strategy: str = 'knn'  # 'mean', 'median', 'knn'
    categorical_imputation_strategy: str = 'mode'
    
    # Scaling
    scaling_method: str = 'robust'  # 'standard', 'robust', 'minmax'
    
    # Calibração
    calibration_method: str = 'isotonic'  # 'sigmoid', 'isotonic'
    
    # Output
    output_dir: Path = field(default_factory=lambda: Path('./models'))
    model_version: str = '10.0.0'
    
    # Compliance
    enable_audit_trail: bool = True
    save_feature_importance: bool = True
    generate_model_card: bool = True
    
    # Random State
    random_state: int = 42
    
    def __post_init__(self):
        """Validações pós-inicialização."""
        # Cria diretório de output se não existir
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Validações básicas
        if self.test_size + self.validation_size >= 0.5:
            raise ValueError("test_size + validation_size deve ser < 0.5")
        
        if self.test_size <= 0 or self.validation_size <= 0:
            raise ValueError("test_size e validation_size devem ser > 0")
        
        if self.n_trials_optuna < 1:
            raise ValueError("n_trials_optuna deve ser >= 1")
        
        if self.oot_months < 1:
            raise ValueError("oot_months deve ser >= 1")

@dataclass
class EnterpriseConfig:
    """Configuração completa enterprise."""
    ml_config: MLConfig = field(default_factory=MLConfig)
    business_metrics: BusinessMetrics = field(default_factory=BusinessMetrics)
    thresholds: ModelThresholds = field(default_factory=ModelThresholds)
    
    def to_dict(self) -> Dict:
        """Serializa configuração para dict."""
        return {
            'ml_config': asdict(self.ml_config),
            'business_metrics': asdict(self.business_metrics),
            'thresholds': asdict(self.thresholds)
        }
    
    def save(self, path: Path):
        """Salva configuração em JSON."""
        with open(path, 'w') as f:
            json.dump(self.to_dict(), f, indent=4, default=str)
        logger.info(f"Configuração salva em {path}")

# ==============================================================================
# 3. VALIDAÇÃO E QUALIDADE DE DADOS
# ==============================================================================

class DataValidator:
    """Validador robusto de qualidade de dados."""
    
    def __init__(self, config: EnterpriseConfig):
        self.config = config
        self.validation_results = {
            'passed': [],
            'warnings': [],
            'errors': []
        }
    
    def validate_all(self, df: pd.DataFrame) -> Tuple[bool, Dict]:
        """Executa todas as validações."""
        logger.info("Iniciando validação de qualidade de dados...")
        
        self._check_target_leakage(df)
        self._check_missing_values(df)
        self._check_data_types(df)
        self._check_value_ranges(df)
        self._check_temporal_consistency(df)
        self._check_class_balance(df)
        
        is_valid = len(self.validation_results['errors']) == 0
        
        self._log_validation_summary()
        
        return is_valid, self.validation_results
    
    def _check_target_leakage(self, df: pd.DataFrame):
        """Detecta potencial data leakage."""
        # Excluir a variável target da verificação de leakage
        target_col = self.config.ml_config.target_variable
        leaked_cols = [col for col in df.columns 
                      if col != target_col and 
                      any(forbidden in col.lower() 
                            for forbidden in FORBIDDEN_FEATURES)]
        
        if leaked_cols:
            self.validation_results['errors'].append({
                'type': DataQualityIssue.TARGET_LEAKAGE,
                'message': f"Colunas com potencial leakage detectadas: {leaked_cols}",
                'severity': AlertLevel.CRITICAL
            })
            logger.critical(f"❌ DATA LEAKAGE DETECTADO: {leaked_cols}")
        else:
            self.validation_results['passed'].append({
                'check': 'target_leakage',
                'message': 'Nenhuma variável com leakage detectada'
            })
            logger.info("✅ Verificação de data leakage: PASSOU")
    
    def _check_missing_values(self, df: pd.DataFrame):
        """Verifica taxa de valores missing."""
        missing_rates = df.isnull().mean()
        high_missing = missing_rates[
            missing_rates > self.config.thresholds.max_feature_missing_rate
        ]
        
        if len(high_missing) > 0:
            self.validation_results['warnings'].append({
                'type': DataQualityIssue.MISSING_VALUES,
                'message': f"Colunas com >30% missing: {high_missing.to_dict()}",
                'severity': AlertLevel.WARNING
            })
            logger.warning(f"⚠️  Colunas com alta taxa de missing: {list(high_missing.index)}")
        else:
            logger.info("✅ Verificação de missing values: PASSOU")
    
    def _check_data_types(self, df: pd.DataFrame):
        """Valida tipos de dados esperados."""
        # Verifica se colunas numéricas estão corretas
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        
        for col in numeric_cols:
            if df[col].dtype == 'object':
                self.validation_results['errors'].append({
                    'type': DataQualityIssue.INVALID_VALUES,
                    'message': f"Coluna {col} deveria ser numérica mas é object",
                    'severity': AlertLevel.CRITICAL
                })
        
        logger.info("✅ Verificação de tipos de dados: PASSOU")
    
    def _check_value_ranges(self, df: pd.DataFrame):
        """Valida ranges de valores."""
        if 'idade' in df.columns:
            invalid_ages = df[
                (df['idade'] < BUSINESS_CONSTANTS['min_age']) | 
                (df['idade'] > BUSINESS_CONSTANTS['max_age'])
            ]
            if len(invalid_ages) > 0:
                self.validation_results['warnings'].append({
                    'type': DataQualityIssue.OUTLIERS,
                    'message': f"{len(invalid_ages)} registros com idade fora do range válido",
                    'severity': AlertLevel.WARNING
                })
        
        logger.info("✅ Verificação de ranges de valores: PASSOU")
    
    def _check_temporal_consistency(self, df: pd.DataFrame):
        """Verifica consistência temporal."""
        date_col = self.config.ml_config.date_column
        
        if date_col in df.columns:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
            future_dates = df[df[date_col] > datetime.now()]
            
            if len(future_dates) > 0:
                self.validation_results['errors'].append({
                    'type': DataQualityIssue.INVALID_VALUES,
                    'message': f"{len(future_dates)} registros com datas futuras",
                    'severity': AlertLevel.CRITICAL
                })
        
        logger.info("✅ Verificação de consistência temporal: PASSOU")
    
    def _check_class_balance(self, df: pd.DataFrame):
        """Verifica desbalanceamento de classes."""
        target_col = self.config.ml_config.target_variable
        
        if target_col in df.columns:
            class_counts = df[target_col].value_counts()
            
            if len(class_counts) < 2:
                self.validation_results['errors'].append({
                    'type': DataQualityIssue.INVALID_VALUES,
                    'message': "Target tem apenas uma classe",
                    'severity': AlertLevel.CRITICAL
                })
                return
            
            imbalance_ratio = class_counts.max() / class_counts.min()
            
            if imbalance_ratio > self.config.thresholds.max_class_imbalance_ratio:
                self.validation_results['warnings'].append({
                    'type': DataQualityIssue.INVALID_VALUES,
                    'message': f"Desbalanceamento alto: ratio {imbalance_ratio:.2f}",
                    'severity': AlertLevel.WARNING
                })
                logger.warning(f"⚠️  Alto desbalanceamento: {imbalance_ratio:.2f}x")
            else:
                logger.info(f"✅ Desbalanceamento aceitável: {imbalance_ratio:.2f}x")
    
    def _log_validation_summary(self):
        """Loga sumário das validações."""
        logger.info("\n" + "="*70)
        logger.info("SUMÁRIO DE VALIDAÇÃO DE DADOS")
        logger.info("="*70)
        logger.info(f"✅ Verificações passou: {len(self.validation_results['passed'])}")
        logger.info(f"⚠️  Warnings: {len(self.validation_results['warnings'])}")
        logger.info(f"❌ Erros: {len(self.validation_results['errors'])}")
        logger.info("="*70 + "\n")

# ==============================================================================
# 4. FEATURE ENGINEERING AVANÇADO
# ==============================================================================

class SmartDateParser:
    """Parser robusto de datas com tratamento de formatos Excel."""
    
    @staticmethod
    def parse_date(value: Any, reference_date: datetime = None) -> pd.Timestamp:
        """
        Parse datas de múltiplos formatos incluindo serial numbers do Excel.
        
        Args:
            value: Valor a ser parseado
            reference_date: Data de referência para validações
        
        Returns:
            pd.Timestamp ou pd.NaT se inválido
        """
        if reference_date is None:
            reference_date = datetime.now()
        
        # Já é timestamp válido
        if isinstance(value, pd.Timestamp):
            return value if value <= reference_date else pd.NaT
        
        # Tenta parse direto
        try:
            date = pd.to_datetime(value, errors='coerce')
            if pd.notna(date) and date <= reference_date:
                return date
        except:
            pass
        
        # Tenta formato serial Excel (Windows)
        if isinstance(value, (int, float)) and not np.isnan(value):
            try:
                # Excel Windows (1900 date system)
                if 1 <= value <= 2958465:  # Range válido Excel
                    date = BUSINESS_CONSTANTS['excel_date_base'] + pd.Timedelta(days=value)
                    
                    # Correção do bug do Excel (1900 não foi bissexto)
                    if value >= 60:
                        date = date - pd.Timedelta(days=1)
                    
                    # Validações de sanidade
                    age = (reference_date - date).days / 365.25
                    if 0 <= age <= 120:  # Idade válida
                        return date
            except:
                pass
        
        return pd.NaT
    
    @staticmethod
    def calculate_age(birth_date: pd.Timestamp, reference_date: datetime = None) -> float:
        """Calcula idade com validações."""
        if reference_date is None:
            reference_date = datetime.now()
        
        if pd.isna(birth_date):
            return np.nan
        
        age = (reference_date - birth_date).days / 365.25
        
        # Validações
        if age < 0 or age > 120:
            return np.nan
        
        return age

class AdvancedFeatureEngineer(BaseEstimator, TransformerMixin):
    """Feature Engineering Enterprise com técnicas avançadas."""
    
    def __init__(self, config: MLConfig):
        self.config = config
        self.feature_stats_ = {}
        self.created_features_ = []
    
    def fit(self, X: pd.DataFrame, y=None):
        """Aprende estatísticas das features."""
        logger.info("Aprendendo estatísticas para feature engineering...")
        
        # Estatísticas para features numéricas
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            self.feature_stats_[col] = {
                'mean': X[col].mean(),
                'median': X[col].median(),
                'std': X[col].std(),
                'q1': X[col].quantile(0.25),
                'q3': X[col].quantile(0.75)
            }
        
        return self
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Aplica engenharia de features."""
        logger.info("Aplicando feature engineering avançado...")
        X_eng = X.copy()
        
        # 1. Tratamento de Datas
        X_eng = self._engineer_date_features(X_eng)
        
        # 2. Features de Razão e Proporção
        X_eng = self._engineer_ratio_features(X_eng)
        
        # 3. Features de Agregação
        X_eng = self._engineer_aggregation_features(X_eng)
        
        # 4. Features de Interação
        if self.config.create_interaction_features:
            X_eng = self._engineer_interaction_features(X_eng)
        
        # 5. Features Polinomiais
        if self.config.create_polynomial_features:
            X_eng = self._engineer_polynomial_features(X_eng)
        
        # 6. Features de Comportamento
        X_eng = self._engineer_behavioral_features(X_eng)
        
        # 7. Features de Risco
        X_eng = self._engineer_risk_features(X_eng)
        
        # Remove infinitos e NaN resultantes
        X_eng = X_eng.replace([np.inf, -np.inf], np.nan)
        
        logger.info(f"✅ Features criadas: {len(self.created_features_)}")
        logger.info(f"   Shape final: {X_eng.shape}")
        
        return X_eng
    
    def _engineer_date_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Processa features de data."""
        if 'nascimento' in X.columns:
            logger.info("   → Processando datas de nascimento...")
            
            # Parse robusto de datas
            birth_dates = X['nascimento'].apply(
                lambda x: SmartDateParser.parse_date(x)
            )
            
            # Calcula idade
            X['idade'] = birth_dates.apply(
                lambda x: SmartDateParser.calculate_age(x)
            )
            
            # Features derivadas de idade
            X['idade_categoria'] = pd.cut(
                X['idade'],
                bins=[0, 25, 35, 50, 65, 100],
                labels=['jovem', 'adulto_jovem', 'adulto', 'senior', 'aposentado']
            )
            
            X['geracao'] = pd.cut(
                X['idade'],
                bins=[0, 28, 44, 60, 100],
                labels=['gen_z', 'millennial', 'gen_x', 'boomer']
            )
            
            # Remove coluna original
            X = X.drop(columns=['nascimento'])
            self.created_features_.extend(['idade', 'idade_categoria', 'geracao'])
        
        # Data de efetivação
        if 'data_efetivacao' in X.columns:
            X['data_efetivacao'] = pd.to_datetime(X['data_efetivacao'], errors='coerce')
            
            X['ano_efetivacao'] = X['data_efetivacao'].dt.year
            X['mes_efetivacao'] = X['data_efetivacao'].dt.month
            X['trimestre_efetivacao'] = X['data_efetivacao'].dt.quarter
            X['dia_semana_efetivacao'] = X['data_efetivacao'].dt.dayofweek
            X['eh_final_de_semana'] = X['dia_semana_efetivacao'].isin([5, 6]).astype(int)
            X['eh_inicio_mes'] = (X['data_efetivacao'].dt.day <= 5).astype(int)
            X['eh_fim_mes'] = (X['data_efetivacao'].dt.day >= 25).astype(int)
            
            self.created_features_.extend([
                'ano_efetivacao', 'mes_efetivacao', 'trimestre_efetivacao',
                'dia_semana_efetivacao', 'eh_final_de_semana',
                'eh_inicio_mes', 'eh_fim_mes'
            ])
        
        return X
    
    def _engineer_ratio_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Cria features de razão."""
        epsilon = 1e-6
        
        # Razão valor/parcelas
        if 'total_financiado' in X.columns and 'quantidade_parcelas' in X.columns:
            X['valor_medio_parcela'] = (
                X['total_financiado'] / (X['quantidade_parcelas'] + epsilon)
            )
            self.created_features_.append('valor_medio_parcela')
        
        # Comprometimento de renda (LTV-like)
        if 'total_financiado' in X.columns and 'renda_declarada' in X.columns:
            X['razao_emprestimo_renda'] = (
                X['total_financiado'] / (X['renda_declarada'] + epsilon)
            )
            
            X['comprometimento_mensal'] = (
                X.get('valor_medio_parcela', 0) / (X['renda_declarada'] + epsilon)
            )
            
            self.created_features_.extend([
                'razao_emprestimo_renda',
                'comprometimento_mensal'
            ])
        
        # Entrada como proporção
        if 'entrada' in X.columns and 'total_financiado' in X.columns:
            X['proporcao_entrada'] = (
                X['entrada'] / (X['total_financiado'] + epsilon)
            )
            self.created_features_.append('proporcao_entrada')
        
        return X
    
    def _engineer_aggregation_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Features de agregação por grupos."""
        
        # Por cidade
        if 'endereco_cidade' in X.columns:
            city_stats = X.groupby('endereco_cidade').agg({
                col: ['mean', 'std', 'count'] 
                for col in X.select_dtypes(include=[np.number]).columns[:3]
            })
            
            # Simplified aggregation
            if 'total_financiado' in X.columns:
                city_avg = X.groupby('endereco_cidade')['total_financiado'].transform('mean')
                X['valor_vs_media_cidade'] = X['total_financiado'] / (city_avg + 1e-6)
                self.created_features_.append('valor_vs_media_cidade')
        
        # Por profissão
        if 'profissao' in X.columns and 'renda_declarada' in X.columns:
            prof_avg_income = X.groupby('profissao')['renda_declarada'].transform('mean')
            X['renda_vs_media_profissao'] = X['renda_declarada'] / (prof_avg_income + 1e-6)
            self.created_features_.append('renda_vs_media_profissao')
        
        return X
    
    def _engineer_interaction_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Features de interação entre variáveis importantes."""
        
        # Idade × Valor do empréstimo
        if 'idade' in X.columns and 'total_financiado' in X.columns:
            X['idade_x_valor'] = X['idade'] * X['total_financiado']
            self.created_features_.append('idade_x_valor')
        
        # Prazo × Valor parcela
        if 'quantidade_parcelas' in X.columns and 'valor_medio_parcela' in X.columns:
            X['prazo_x_parcela'] = X['quantidade_parcelas'] * X['valor_medio_parcela']
            self.created_features_.append('prazo_x_parcela')
        
        return X
    
    def _engineer_polynomial_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Features polinomiais para capturar não-linearidades."""
        
        poly_candidates = ['idade', 'total_financiado', 'quantidade_parcelas']
        poly_candidates = [c for c in poly_candidates if c in X.columns]
        
        for col in poly_candidates[:2]:  # Limita para evitar explosão dimensional
            if X[col].dtype in [np.float64, np.int64]:
                X[f'{col}_squared'] = X[col] ** 2
                X[f'{col}_sqrt'] = np.sqrt(np.abs(X[col]))
                self.created_features_.extend([f'{col}_squared', f'{col}_sqrt'])
        
        return X
    
    def _engineer_behavioral_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Features comportamentais."""
        
        # Flag cliente novo
        if 'quantidade_parcelas' in X.columns:
            X['eh_cliente_novo'] = (X['quantidade_parcelas'] <= 2).astype(int)
            self.created_features_.append('eh_cliente_novo')
        
        # Perfil de risco baseado em múltiplas variáveis
        if 'idade' in X.columns and 'total_financiado' in X.columns:
            # Cliente jovem + valor alto = maior risco
            X['perfil_risco_idade_valor'] = (
                (X['idade'] < 25) & (X['total_financiado'] > X['total_financiado'].quantile(0.75))
            ).astype(int)
            self.created_features_.append('perfil_risco_idade_valor')
        
        return X
    
    def _engineer_risk_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Features específicas para scoring de risco."""
        
        # Prazo longo = maior risco
        if 'quantidade_parcelas' in X.columns:
            X['eh_prazo_longo'] = (X['quantidade_parcelas'] > 48).astype(int)
            self.created_features_.append('eh_prazo_longo')
        
        # Comprometimento alto
        if 'comprometimento_mensal' in X.columns:
            X['comprometimento_alto'] = (X['comprometimento_mensal'] > 0.3).astype(int)
            self.created_features_.append('comprometimento_alto')
        
        return X

# ==============================================================================
# 5. PIPELINE DE PRÉ-PROCESSAMENTO ROBUSTO
# ==============================================================================

class RobustOutlierTransformer(BaseEstimator, TransformerMixin):
    """Tratamento robusto de outliers usando IQR."""
    
    def __init__(self, factor=1.5, method='winsorize'):
        self.factor = factor
        self.method = method  # 'winsorize', 'clip', 'remove'
        self.bounds_ = {}
    
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X)
        
        for col in X_df.columns:
            if X_df[col].dtype in [np.float64, np.int64]:
                Q1 = X_df[col].quantile(0.25)
                Q3 = X_df[col].quantile(0.75)
                IQR = Q3 - Q1
                
                self.bounds_[col] = {
                    'lower': Q1 - self.factor * IQR,
                    'upper': Q3 + self.factor * IQR
                }
        
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X).copy()
        
        for col, bounds in self.bounds_.items():
            if col in X_df.columns:
                if self.method == 'winsorize':
                    X_df[col] = X_df[col].clip(
                        lower=bounds['lower'],
                        upper=bounds['upper']
                    )
        
        return X_df.values

class SmartImputer(BaseEstimator, TransformerMixin):
    """Imputador inteligente que escolhe estratégia por coluna."""
    
    def __init__(self, numeric_strategy='knn', categorical_strategy='mode'):
        self.numeric_strategy = numeric_strategy
        self.categorical_strategy = categorical_strategy
        self.imputers_ = {}
    
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X)
        
        # Imputer numérico
        numeric_cols = X_df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            if self.numeric_strategy == 'knn':
                self.imputers_['numeric'] = KNNImputer(n_neighbors=5)
            else:
                self.imputers_['numeric'] = SimpleImputer(strategy=self.numeric_strategy)
            
            self.imputers_['numeric'].fit(X_df[numeric_cols])
        
        # Imputer categórico
        categorical_cols = X_df.select_dtypes(include=['object', 'category']).columns
        if len(categorical_cols) > 0:
            self.imputers_['categorical'] = SimpleImputer(
                strategy='most_frequent'
            )
            self.imputers_['categorical'].fit(X_df[categorical_cols])
        
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X).copy()
        
        # Imputa numéricos
        numeric_cols = X_df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0 and 'numeric' in self.imputers_:
            X_df[numeric_cols] = self.imputers_['numeric'].transform(X_df[numeric_cols])
        
        # Imputa categóricos
        categorical_cols = X_df.select_dtypes(include=['object', 'category']).columns
        if len(categorical_cols) > 0 and 'categorical' in self.imputers_:
            X_df[categorical_cols] = self.imputers_['categorical'].transform(X_df[categorical_cols])
        
        return X_df

def build_preprocessing_pipeline(config: MLConfig) -> Pipeline:
    """Constrói pipeline de pré-processamento robusto."""
    
    steps = [
        ('feature_engineering', AdvancedFeatureEngineer(config)),
        ('outlier_treatment', RobustOutlierTransformer(factor=1.5)),
        ('imputation', SmartImputer(
            numeric_strategy=config.numeric_imputation_strategy,
            categorical_strategy=config.categorical_imputation_strategy
        ))
    ]
    
    # Adiciona scaler se necessário
    if config.scaling_method == 'robust':
        steps.append(('scaling', RobustScaler()))
    elif config.scaling_method == 'standard':
        steps.append(('scaling', StandardScaler()))
    
    return Pipeline(steps)

# ==============================================================================
# 6. GESTÃO DE DADOS TEMPORAIS
# ==============================================================================

class TemporalDataSplitter:
    """Split temporal para evitar data leakage."""
    
    def __init__(self, config: MLConfig):
        self.config = config
    
    def split(self, X: pd.DataFrame, y: pd.Series) -> Dict[str, Tuple]:
        """
        Realiza split temporal garantindo que:
        - Train: dados mais antigos
        - Validation: período intermediário
        - Test: período recente
        - OOT: últimos N meses (completamente fora da amostra)
        """
        logger.info("Realizando split temporal dos dados...")
        
        date_col = self.config.date_column
        
        if date_col not in X.columns:
            logger.warning(f"Coluna de data '{date_col}' não encontrada. Usando split aleatório estratificado.")
            return self._random_stratified_split(X, y)
        
        # Ordena por data
        X_sorted = X.sort_values(date_col).reset_index(drop=True)
        y_sorted = y.loc[X_sorted.index].reset_index(drop=True)
        
        # Define cutoffs temporais
        max_date = X_sorted[date_col].max()
        
        # OOT: últimos N meses
        oot_cutoff = max_date - pd.DateOffset(months=self.config.oot_months)
        
        oot_mask = X_sorted[date_col] >= oot_cutoff
        train_val_test_mask = ~oot_mask
        
        # Separa OOT
        X_oot = X_sorted[oot_mask].reset_index(drop=True)
        y_oot = y_sorted[oot_mask].reset_index(drop=True)
        
        X_remaining = X_sorted[train_val_test_mask].reset_index(drop=True)
        y_remaining = y_sorted[train_val_test_mask].reset_index(drop=True)
        
        # Split do restante
        n_remaining = len(X_remaining)
        n_test = int(n_remaining * self.config.test_size)
        n_val = int(n_remaining * self.config.validation_size)
        n_train = n_remaining - n_test - n_val
        
        X_train = X_remaining.iloc[:n_train]
        y_train = y_remaining.iloc[:n_train]
        
        X_val = X_remaining.iloc[n_train:n_train+n_val]
        y_val = y_remaining.iloc[n_train:n_train+n_val]
        
        X_test = X_remaining.iloc[n_train+n_val:]
        y_test = y_remaining.iloc[n_train+n_val:]
        
        logger.info(f"✅ Split temporal realizado:")
        logger.info(f"   Train: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
        logger.info(f"   Validation: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
        logger.info(f"   Test: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
        logger.info(f"   OOT: {len(X_oot)} ({len(X_oot)/len(X)*100:.1f}%)")
        
        return {
            'train': (X_train, y_train),
            'validation': (X_val, y_val),
            'test': (X_test, y_test),
            'oot': (X_oot, y_oot)
        }
    
    def _random_stratified_split(self, X: pd.DataFrame, y: pd.Series) -> Dict[str, Tuple]:
        """Fallback para split estratificado aleatório."""
        from sklearn.model_selection import train_test_split
        
        # Train + Val vs Test + OOT
        X_train_val, X_test_oot, y_train_val, y_test_oot = train_test_split(
            X, y,
            test_size=self.config.test_size + self.config.validation_size + 0.10,
            stratify=y,
            random_state=self.config.random_state
        )
        
        # Train vs Val
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_val, y_train_val,
            test_size=self.config.validation_size / (1 - self.config.test_size - 0.10),
            stratify=y_train_val,
            random_state=self.config.random_state
        )
        
        # Test vs OOT
        X_test, X_oot, y_test, y_oot = train_test_split(
            X_test_oot, y_test_oot,
            test_size=0.33,
            stratify=y_test_oot,
            random_state=self.config.random_state
        )
        
        return {
            'train': (X_train, y_train),
            'validation': (X_val, y_val),
            'test': (X_test, y_test),
            'oot': (X_oot, y_oot)
        }

# ==============================================================================
# 7. MÉTRICAS E AVALIAÇÃO ENTERPRISE
# ==============================================================================

class CreditScoringMetrics:
    """Calculadora completa de métricas para credit scoring."""
    
    @staticmethod
    def calculate_all_metrics(y_true: np.ndarray, 
                             y_prob: np.ndarray,
                             y_pred: np.ndarray = None,
                             threshold: float = 0.5) -> Dict[str, float]:
        """Calcula todas as métricas relevantes."""
        
        if y_pred is None:
            y_pred = (y_prob >= threshold).astype(int)
        
        # Métricas de discriminação
        auc = roc_auc_score(y_true, y_prob)
        gini = 2 * auc - 1
        
        # KS Statistic
        ks_stat, ks_pvalue = ks_2samp(
            y_prob[y_true == 0],
            y_prob[y_true == 1]
        )
        
        # Métricas de calibração
        brier = brier_score_loss(y_true, y_prob)
        logloss = log_loss(y_true, y_prob)
        
        # Confusion Matrix
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        # Métricas de classificação
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        # Métricas de negócio
        approval_rate = (y_pred == 0).mean()  # Assumindo 0 = aprovado
        bad_rate = y_true[y_pred == 0].mean() if sum(y_pred == 0) > 0 else 0
        
        # Average Precision (útil para dados desbalanceados)
        avg_precision = average_precision_score(y_true, y_prob)
        
        return {
            # Discriminação
            'auc': auc,
            'gini': gini,
            'ks_statistic': ks_stat,
            'ks_pvalue': ks_pvalue,
            
            # Calibração
            'brier_score': brier,
            'log_loss': logloss,
            
            # Classificação
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'specificity': specificity,
            'avg_precision': avg_precision,
            
            # Confusion Matrix
            'true_negatives': int(tn),
            'false_positives': int(fp),
            'false_negatives': int(fn),
            'true_positives': int(tp),
            
            # Negócio
            'approval_rate': approval_rate,
            'bad_rate': bad_rate,
            'threshold': threshold
        }
    
    @staticmethod
    def calculate_business_value(metrics: Dict[str, float],
                                business_config: BusinessMetrics) -> Dict[str, float]:
        """Calcula valor de negócio do modelo."""
        
        tn = metrics['true_negatives']
        fp = metrics['false_positives']
        fn = metrics['false_negatives']
        tp = metrics['true_positives']
        
        # Receita de clientes bons aprovados
        revenue_good_clients = tn * business_config.revenue_per_good_client
        
        # Perda de clientes maus aprovados
        loss_bad_clients = fp * business_config.loss_per_bad_client
        
        # Custo operacional
        total_analyzed = tn + fp + fn + tp
        operational_cost = total_analyzed * business_config.operational_cost_per_analysis
        
        # Profit total
        total_profit = revenue_good_clients - loss_bad_clients - operational_cost
        
        # Profit por cliente analisado
        profit_per_client = total_profit / total_analyzed if total_analyzed > 0 else 0
        
        # ROI
        roi = (total_profit / operational_cost - 1) * 100 if operational_cost > 0 else 0
        
        return {
            'total_revenue': revenue_good_clients,
            'total_losses': loss_bad_clients,
            'operational_cost': operational_cost,
            'net_profit': total_profit,
            'profit_per_client': profit_per_client,
            'roi_percentage': roi,
            'clients_approved': tn + fp,
            'clients_rejected': fn + tp
        }

class PSICalculator:
    """Calculador de Population Stability Index (PSI)."""
    
    @staticmethod
    def calculate_psi(expected: np.ndarray, 
                     actual: np.ndarray,
                     buckets: int = 10,
                     epsilon: float = 0.0001) -> Tuple[float, pd.DataFrame]:
        """
        Calcula PSI entre duas distribuições.
        
        Interpretação:
        - PSI < 0.10: Sem mudança significativa
        - 0.10 <= PSI < 0.25: Mudança moderada, investigar
        - PSI >= 0.25: Mudança significativa, retreino necessário
        """
        
        # Remove NaN
        expected = expected[~np.isnan(expected)]
        actual = actual[~np.isnan(actual)]
        
        # Define bins baseado na distribuição esperada
        breakpoints = np.percentile(expected, np.linspace(0, 100, buckets + 1))
        breakpoints = np.unique(breakpoints)  # Remove duplicatas
        
        # Conta elementos em cada bucket
        expected_percents = np.histogram(expected, bins=breakpoints)[0] / len(expected)
        actual_percents = np.histogram(actual, bins=breakpoints)[0] / len(actual)
        
        # Adiciona epsilon para evitar log(0)
        expected_percents = np.clip(expected_percents, epsilon, 1)
        actual_percents = np.clip(actual_percents, epsilon, 1)
        
        # Calcula PSI
        psi_values = (actual_percents - expected_percents) * np.log(actual_percents / expected_percents)
        psi_total = np.sum(psi_values)
        
        # DataFrame com detalhes
        psi_df = pd.DataFrame({
            'bucket': range(len(psi_values)),
            'expected_percent': expected_percents,
            'actual_percent': actual_percents,
            'psi_contribution': psi_values
        })
        
        return psi_total, psi_df
    
    @staticmethod
    def calculate_feature_psi(X_expected: pd.DataFrame,
                             X_actual: pd.DataFrame,
                             buckets: int = 10) -> pd.DataFrame:
        """Calcula PSI para todas as features."""
        
        psi_results = []
        
        numeric_cols = X_expected.select_dtypes(include=[np.number]).columns
        
        for col in numeric_cols:
            try:
                psi, _ = PSICalculator.calculate_psi(
                    X_expected[col].values,
                    X_actual[col].values,
                    buckets=buckets
                )
                
                psi_results.append({
                    'feature': col,
                    'psi': psi,
                    'status': 'OK' if psi < 0.10 else ('WARNING' if psi < 0.25 else 'CRITICAL')
                })
            except Exception as e:
                logger.warning(f"Erro ao calcular PSI para {col}: {e}")
        
        return pd.DataFrame(psi_results).sort_values('psi', ascending=False)

# ==============================================================================
# 8. OTIMIZAÇÃO DE HIPERPARÂMETROS INTELIGENTE
# ==============================================================================

class BayesianOptimizer:
    """Otimizador Bayesiano com Optuna para múltiplos modelos."""
    
    def __init__(self, config: EnterpriseConfig):
        self.config = config
        self.study = None
        self.best_model_type = None
    
    def optimize(self,
                X_train: pd.DataFrame,
                y_train: pd.Series,
                X_val: pd.DataFrame,
                y_val: pd.Series) -> Dict[str, Any]:
        """Otimiza hiperparâmetros para múltiplos modelos."""
        
        logger.info("="*70)
        logger.info("INICIANDO OTIMIZAÇÃO BAYESIANA DE HIPERPARÂMETROS")
        logger.info("="*70)
        
        # Cria estudo Optuna
        self.study = optuna.create_study(
            direction='maximize',
            study_name=f'credit_scoring_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
            sampler=optuna.samplers.TPESampler(seed=self.config.ml_config.random_state)
        )
        
        # Função objetivo que será otimizada
        def objective(trial: optuna.Trial) -> float:
            # Escolhe tipo de modelo
            model_type = trial.suggest_categorical(
                'model_type',
                self.config.ml_config.models_to_evaluate
            )
            
            # Obtém hiperparâmetros específicos do modelo
            if model_type == 'xgboost':
                params = self._get_xgboost_params(trial, y_train)
                model = xgb.XGBClassifier(**params)
            
            elif model_type == 'lightgbm':
                params = self._get_lightgbm_params(trial, y_train)
                model = lgb.LGBMClassifier(**params)
            
            elif model_type == 'ensemble':
                # Ensemble de XGBoost + LightGBM
                xgb_params = self._get_xgboost_params(trial, y_train, prefix='xgb_')
                lgb_params = self._get_lightgbm_params(trial, y_train, prefix='lgb_')
                
                model = VotingEnsemble(
                    xgb.XGBClassifier(**xgb_params),
                    lgb.LGBMClassifier(**lgb_params),
                    weights=trial.suggest_float('ensemble_weight_xgb', 0.3, 0.7)
                )
            
            # Treina modelo
            try:
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_val, y_val)] if hasattr(model, 'eval_set') else None,
                    verbose=False
                )
                
                # Predição
                y_prob = model.predict_proba(X_val)[:, 1]
                
                # Métrica objetivo: AUC
                auc = roc_auc_score(y_val, y_prob)
                
                # Log progresso
                trial_num = trial.number + 1
                total_trials = self.config.ml_config.n_trials_optuna
                logger.info(f"Trial {trial_num}/{total_trials} | Modelo: {model_type} | AUC: {auc:.4f}")
                
                return auc
                
            except Exception as e:
                logger.error(f"Erro no trial {trial.number}: {e}")
                return 0.0
        
        # Executa otimização
        self.study.optimize(
            objective,
            n_trials=self.config.ml_config.n_trials_optuna,
            timeout=self.config.ml_config.optuna_timeout,
            show_progress_bar=True,
            callbacks=[self._logging_callback]
        )
        
        logger.info("="*70)
        logger.info(f"✅ OTIMIZAÇÃO CONCLUÍDA")
        logger.info(f"   Melhor AUC: {self.study.best_value:.4f}")
        logger.info(f"   Melhor modelo: {self.study.best_params.get('model_type', 'N/A')}")
        logger.info("="*70)
        
        return {
            'best_params': self.study.best_params,
            'best_value': self.study.best_value,
            'best_trial': self.study.best_trial.number,
            'optimization_history': self._get_optimization_history()
        }
    
    def _get_xgboost_params(self, trial: optuna.Trial, y_train: pd.Series, prefix: str = '') -> Dict:
        """Define espaço de busca para XGBoost."""
        
        # Calcula scale_pos_weight para desbalanceamento
        class_counts = y_train.value_counts()
        scale_pos_weight = class_counts[0] / class_counts[1]
        
        return {
            'n_estimators': trial.suggest_int(f'{prefix}n_estimators', 100, 1000, step=100),
            'max_depth': trial.suggest_int(f'{prefix}max_depth', 3, 10),
            'learning_rate': trial.suggest_float(f'{prefix}learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float(f'{prefix}subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float(f'{prefix}colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int(f'{prefix}min_child_weight', 1, 10),
            'gamma': trial.suggest_float(f'{prefix}gamma', 0, 5),
            'reg_alpha': trial.suggest_float(f'{prefix}reg_alpha', 0, 5),
            'reg_lambda': trial.suggest_float(f'{prefix}reg_lambda', 0, 5),
            'scale_pos_weight': scale_pos_weight,
            'random_state': self.config.ml_config.random_state,
            'n_jobs': -1,
            'tree_method': 'hist',
            'enable_categorical': True
        }
    
    def _get_lightgbm_params(self, trial: optuna.Trial, y_train: pd.Series, prefix: str = '') -> Dict:
        """Define espaço de busca para LightGBM."""
        
        class_counts = y_train.value_counts()
        scale_pos_weight = class_counts[0] / class_counts[1]
        
        return {
            'n_estimators': trial.suggest_int(f'{prefix}n_estimators', 100, 1000, step=100),
            'max_depth': trial.suggest_int(f'{prefix}max_depth', 3, 10),
            'learning_rate': trial.suggest_float(f'{prefix}learning_rate', 0.01, 0.3, log=True),
            'num_leaves': trial.suggest_int(f'{prefix}num_leaves', 20, 150),
            'subsample': trial.suggest_float(f'{prefix}subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float(f'{prefix}colsample_bytree', 0.6, 1.0),
            'min_child_samples': trial.suggest_int(f'{prefix}min_child_samples', 5, 100),
            'reg_alpha': trial.suggest_float(f'{prefix}reg_alpha', 0, 5),
            'reg_lambda': trial.suggest_float(f'{prefix}reg_lambda', 0, 5),
            'scale_pos_weight': scale_pos_weight,
            'random_state': self.config.ml_config.random_state,
            'n_jobs': -1,
            'verbose': -1
        }
    
    def _logging_callback(self, study: optuna.Study, trial: optuna.Trial):
        """Callback para logging durante otimização."""
        if trial.number % 10 == 0:
            logger.info(f"Progress: {trial.number}/{self.config.ml_config.n_trials_optuna} trials completed")
    
    def _get_optimization_history(self) -> pd.DataFrame:
        """Retorna histórico de otimização."""
        return self.study.trials_dataframe()

class VotingEnsemble:
    """Ensemble simples de modelos com voting."""
    
    def __init__(self, model1, model2, weights: float = 0.5):
        self.model1 = model1
        self.model2 = model2
        self.weight1 = weights
        self.weight2 = 1 - weights
    
    def fit(self, X, y, **kwargs):
        self.model1.fit(X, y)
        self.model2.fit(X, y)
        return self
    
    def predict_proba(self, X):
        prob1 = self.model1.predict_proba(X)
        prob2 = self.model2.predict_proba(X)
        return self.weight1 * prob1 + self.weight2 * prob2
    
    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)

# ==============================================================================
# 9. OTIMIZAÇÃO DE CUTOFF BASEADA EM NEGÓCIO
# ==============================================================================

class CutoffOptimizer:
    """Otimiza threshold de decisão baseado em métricas de negócio."""
    
    def __init__(self, business_config: BusinessMetrics):
        self.business_config = business_config
        self.optimal_cutoff = None
        self.cutoff_analysis = None
    
    def find_optimal_cutoff(self,
                           y_true: np.ndarray,
                           y_prob: np.ndarray,
                           strategy: str = 'profit') -> float:
        """
        Encontra cutoff ótimo baseado em estratégia de negócio.
        
        Estratégias:
        - 'profit': Maximiza lucro
        - 'f1': Maximiza F1-score
        - 'target_bad_rate': Atinge bad rate alvo
        - 'max_approval': Maximiza aprovações mantendo bad rate aceitável
        """
        
        logger.info(f"Otimizando cutoff usando estratégia: {strategy}")
        
        thresholds = np.linspace(0.05, 0.95, 200)
        results = []
        
        for threshold in thresholds:
            y_pred = (y_prob >= threshold).astype(int)
            
            # Calcula métricas
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
            
            # Métricas de classificação
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            # Métricas de negócio
            approval_rate = (y_pred == 0).mean()
            bad_rate = fp / (tn + fp) if (tn + fp) > 0 else 0
            
            # Cálculo de profit
            revenue = tn * self.business_config.revenue_per_good_client
            losses = fp * self.business_config.loss_per_bad_client
            costs = (tn + fp + fn + tp) * self.business_config.operational_cost_per_analysis
            profit = revenue - losses - costs
            
            results.append({
                'threshold': threshold,
                'profit': profit,
                'f1_score': f1,
                'approval_rate': approval_rate,
                'bad_rate': bad_rate,
                'precision': precision,
                'recall': recall,
                'true_negatives': tn,
                'false_positives': fp,
                'false_negatives': fn,
                'true_positives': tp
            })
        
        self.cutoff_analysis = pd.DataFrame(results)
        
        # Seleciona cutoff baseado na estratégia
        if strategy == 'profit':
            # Respeita max_approval_rate
            valid = self.cutoff_analysis[
                self.cutoff_analysis['approval_rate'] <= self.business_config.max_approval_rate
            ]
            if len(valid) > 0:
                optimal_row = valid.loc[valid['profit'].idxmax()]
            else:
                optimal_row = self.cutoff_analysis.loc[self.cutoff_analysis['profit'].idxmax()]
        
        elif strategy == 'f1':
            optimal_row = self.cutoff_analysis.loc[self.cutoff_analysis['f1_score'].idxmax()]
        
        elif strategy == 'target_bad_rate':
            # Encontra cutoff que chega mais próximo do bad_rate alvo
            self.cutoff_analysis['distance_to_target'] = np.abs(
                self.cutoff_analysis['bad_rate'] - self.business_config.target_bad_rate
            )
            optimal_row = self.cutoff_analysis.loc[
                self.cutoff_analysis['distance_to_target'].idxmin()
            ]
        
        elif strategy == 'max_approval':
            # Máxima aprovação mantendo bad_rate <= target
            valid = self.cutoff_analysis[
                self.cutoff_analysis['bad_rate'] <= self.business_config.target_bad_rate
            ]
            if len(valid) > 0:
                optimal_row = valid.loc[valid['approval_rate'].idxmax()]
            else:
                optimal_row = self.cutoff_analysis.loc[self.cutoff_analysis['approval_rate'].idxmax()]
        
        else:
            raise ValueError(f"Estratégia desconhecida: {strategy}")
        
        self.optimal_cutoff = optimal_row['threshold']
        
        logger.info(f"✅ Cutoff ótimo encontrado: {self.optimal_cutoff:.4f}")
        logger.info(f"   Approval Rate: {optimal_row['approval_rate']:.2%}")
        logger.info(f"   Bad Rate: {optimal_row['bad_rate']:.2%}")
        logger.info(f"   Profit: R$ {optimal_row['profit']:,.2f}")
        
        return self.optimal_cutoff
    
    def plot_cutoff_analysis(self, save_path: Optional[Path] = None):
        """Plota análise de cutoff."""
        
        if self.cutoff_analysis is None:
            logger.warning("Execute find_optimal_cutoff primeiro")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Profit vs Threshold
        axes[0, 0].plot(self.cutoff_analysis['threshold'], self.cutoff_analysis['profit'])
        axes[0, 0].axvline(self.optimal_cutoff, color='r', linestyle='--', label='Optimal Cutoff')
        axes[0, 0].set_xlabel('Threshold')
        axes[0, 0].set_ylabel('Profit (R$)')
        axes[0, 0].set_title('Profit vs Threshold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Approval Rate vs Bad Rate
        axes[0, 1].plot(self.cutoff_analysis['approval_rate'], self.cutoff_analysis['bad_rate'])
        axes[0, 1].axhline(self.business_config.target_bad_rate, color='g', linestyle='--', 
                          label='Target Bad Rate')
        axes[0, 1].axvline(self.business_config.max_approval_rate, color='orange', linestyle='--',
                          label='Max Approval Rate')
        axes[0, 1].set_xlabel('Approval Rate')
        axes[0, 1].set_ylabel('Bad Rate')
        axes[0, 1].set_title('Approval Rate vs Bad Rate Trade-off')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # F1 Score vs Threshold
        axes[1, 0].plot(self.cutoff_analysis['threshold'], self.cutoff_analysis['f1_score'])
        axes[1, 0].axvline(self.optimal_cutoff, color='r', linestyle='--', label='Optimal Cutoff')
        axes[1, 0].set_xlabel('Threshold')
        axes[1, 0].set_ylabel('F1 Score')
        axes[1, 0].set_title('F1 Score vs Threshold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Precision-Recall Trade-off
        axes[1, 1].plot(self.cutoff_analysis['recall'], self.cutoff_analysis['precision'])
        axes[1, 1].set_xlabel('Recall')
        axes[1, 1].set_ylabel('Precision')
        axes[1, 1].set_title('Precision-Recall Curve')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            logger.info(f"Análise de cutoff salva em: {save_path}")
        
        plt.show()

# ==============================================================================
# 10. SISTEMA DE MONITORAMENTO E ALERTAS
# ==============================================================================

class ModelMonitor:
    """Sistema de monitoramento contínuo de performance."""
    
    def __init__(self, config: EnterpriseConfig, baseline_metrics: Dict):
        self.config = config
        self.baseline_metrics = baseline_metrics
        self.performance_history = []
        self.alerts = []
    
    def evaluate_batch(self,
                      model,
                      X: pd.DataFrame,
                      y_true: pd.Series,
                      batch_date: datetime,
                      X_baseline: pd.DataFrame) -> Dict[str, Any]:
        """Avalia batch de produção e detecta anomalias."""
        
        logger.info(f"Monitorando batch de {batch_date.strftime('%Y-%m-%d')}...")
        
        # Predições
        y_prob = model.predict_proba(X)[:, 1]
        y_pred = (y_prob >= self.config.thresholds.min_auc_acceptable).astype(int)
        
        # Calcula métricas atuais
        current_metrics = CreditScoringMetrics.calculate_all_metrics(
            y_true.values, y_prob, y_pred
        )
        current_metrics['batch_date'] = batch_date
        current_metrics['sample_size'] = len(X)
        
        # Detecta degradação de performance
        self._check_performance_degradation(current_metrics)
        
        # Detecta data drift (PSI)
        self._check_data_drift(X, X_baseline)
        
        # Detecta concept drift
        self._check_concept_drift(current_metrics)
        
        # Registra histórico
        self.performance_history.append(current_metrics)
        
        # Gera relatório
        report = self._generate_monitoring_report(current_metrics)
        
        return report
    
    def _check_performance_degradation(self, current_metrics: Dict):
        """Detecta degradação de performance."""
        
        auc_drop = self.baseline_metrics['auc'] - current_metrics['auc']
        
        if auc_drop > 0.10:
            self._create_alert(
                AlertLevel.CRITICAL,
                f"AUC caiu {auc_drop:.3f} pontos! "
                f"Baseline: {self.baseline_metrics['auc']:.4f}, "
                f"Atual: {current_metrics['auc']:.4f}"
            )
        elif auc_drop > 0.05:
            self._create_alert(
                AlertLevel.WARNING,
                f"AUC caiu {auc_drop:.3f} pontos. Monitorar de perto."
            )
        
        # Verifica outras métricas críticas
        if current_metrics['auc'] < self.config.thresholds.min_auc_acceptable:
            self._create_alert(
                AlertLevel.EMERGENCY,
                f"AUC abaixo do mínimo aceitável! "
                f"Atual: {current_metrics['auc']:.4f}, "
                f"Mínimo: {self.config.thresholds.min_auc_acceptable:.4f}"
            )
        
        if current_metrics['ks_statistic'] < self.config.thresholds.min_ks_acceptable:
            self._create_alert(
                AlertLevel.CRITICAL,
                f"KS Statistic abaixo do aceitável: {current_metrics['ks_statistic']:.4f}"
            )
    
    def _check_data_drift(self, X_current: pd.DataFrame, X_baseline: pd.DataFrame):
        """Detecta data drift usando PSI."""
        
        feature_psi = PSICalculator.calculate_feature_psi(X_baseline, X_current)
        
        # Features com PSI crítico
        critical_features = feature_psi[feature_psi['status'] == 'CRITICAL']
        
        if len(critical_features) > 0:
            self._create_alert(
                AlertLevel.CRITICAL,
                f"Data drift crítico detectado em {len(critical_features)} features: "
                f"{critical_features['feature'].tolist()}"
            )
        
        # Features com PSI warning
        warning_features = feature_psi[feature_psi['status'] == 'WARNING']
        
        if len(warning_features) > 0:
            self._create_alert(
                AlertLevel.WARNING,
                f"Data drift moderado em {len(warning_features)} features: "
                f"{warning_features['feature'].tolist()}"
            )
    
    def _check_concept_drift(self, current_metrics: Dict):
        """Detecta concept drift (mudança na relação X->y)."""
        
        if len(self.performance_history) < 5:
            return  # Precisa de histórico
        
        # Analisa tendência de AUC nos últimos 5 batches
        recent_aucs = [m['auc'] for m in self.performance_history[-5:]]
        
        # Regressão linear simples para detectar tendência
        x = np.arange(len(recent_aucs))
        slope = np.polyfit(x, recent_aucs, 1)[0]
        
        if slope < -0.01:  # Queda de >1% por batch
            self._create_alert(
                AlertLevel.WARNING,
                f"Tendência de queda na performance detectada. "
                f"Slope: {slope:.4f}"
            )
    
    def _create_alert(self, level: AlertLevel, message: str):
        """Cria alerta."""
        
        alert = {
            'timestamp': datetime.now(),
            'level': level.value,
            'message': message
        }
        
        self.alerts.append(alert)
        
        # Log baseado no nível
        if level == AlertLevel.EMERGENCY or level == AlertLevel.CRITICAL:
            logger.critical(f"🚨 {message}")
        elif level == AlertLevel.WARNING:
            logger.warning(f"⚠️  {message}")
        else:
            logger.info(f"ℹ️  {message}")
    
    def _generate_monitoring_report(self, current_metrics: Dict) -> Dict:
        """Gera relatório de monitoramento."""
        
        return {
            'current_metrics': current_metrics,
            'baseline_metrics': self.baseline_metrics,
            'alerts': self.alerts[-10:],  # Últimos 10 alertas
            'performance_trend': self._calculate_trend(),
            'recommendation': self._get_recommendation()
        }
    
    def _calculate_trend(self) -> str:
        """Calcula tendência de performance."""
        
        if len(self.performance_history) < 3:
            return "INSUFFICIENT_DATA"
        
        recent_aucs = [m['auc'] for m in self.performance_history[-5:]]
        
        if len(recent_aucs) < 2:
            return "STABLE"
        
        slope = np.polyfit(range(len(recent_aucs)), recent_aucs, 1)[0]
        
        if slope < -0.005:
            return "DECLINING"
        elif slope > 0.005:
            return "IMPROVING"
        else:
            return "STABLE"
    
    def _get_recommendation(self) -> str:
        """Retorna recomendação baseada em alertas."""
        
        recent_alerts = self.alerts[-10:]
        
        critical_count = sum(1 for a in recent_alerts if a['level'] in ['critical', 'emergency'])
        
        if critical_count >= 3:
            return "IMMEDIATE_RETRAINING_REQUIRED"
        elif critical_count >= 1:
            return "SCHEDULE_RETRAINING"
        elif len([a for a in recent_alerts if a['level'] == 'warning']) >= 5:
            return "MONITOR_CLOSELY"
        else:
            return "CONTINUE_MONITORING"

# ==============================================================================
# 11. EXPLICABILIDADE COM SHAP
# ==============================================================================

class ModelExplainer:
    """Gerador de explicações do modelo usando SHAP."""
    
    def __init__(self, model, X_background: pd.DataFrame, max_samples: int = 100):
        """
        Args:
            model: Modelo treinado
            X_background: Amostra de dados para background (treino)
            max_samples: Máximo de amostras para usar como background
        """
        self.model = model
        self.X_background = X_background.sample(
            n=min(max_samples, len(X_background)),
            random_state=42
        )
        self.explainer = None
        self.shap_values = None
    
    def compute_shap_values(self, X: pd.DataFrame):
        """Calcula SHAP values."""
        
        logger.info("Calculando SHAP values para explicabilidade...")
        
        # Cria explainer
        self.explainer = shap.TreeExplainer(
            self.model.named_steps['classifier'],
            self.X_background
        )
        
        # Calcula SHAP values
        self.shap_values = self.explainer.shap_values(X)
        
        logger.info("✅ SHAP values calculados")
        
        return self.shap_values
    
    def plot_summary(self, X: pd.DataFrame, save_path: Optional[Path] = None):
        """Plota summary plot do SHAP."""
        
        if self.shap_values is None:
            self.compute_shap_values(X)
        
        plt.figure(figsize=(12, 8))
        shap.summary_plot(
            self.shap_values,
            X,
            plot_type="bar",
            show=False
        )
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            logger.info(f"SHAP summary plot salvo em: {save_path}")
        
        plt.show()
    
    def get_feature_importance(self, X: pd.DataFrame) -> pd.DataFrame:
        """Retorna importância de features via SHAP."""
        
        if self.shap_values is None:
            self.compute_shap_values(X)
        
        importance = np.abs(self.shap_values).mean(axis=0)
        
        importance_df = pd.DataFrame({
            'feature': X.columns,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        return importance_df
    
    def explain_prediction(self, instance: pd.DataFrame, instance_idx: int = 0):
        """Explica uma predição específica."""
        
        if self.shap_values is None:
            self.compute_shap_values(instance)
        
        shap.force_plot(
            self.explainer.expected_value,
            self.shap_values[instance_idx],
            instance.iloc[instance_idx],
            matplotlib=True
        )

# ==============================================================================
# 12. ARTEFATOS E PERSISTÊNCIA ENTERPRISE
# ==============================================================================

class ModelArtifact:
    """Gerenciador de artefatos do modelo (enterprise-grade)."""
    
    def __init__(self,
                 model: Pipeline,
                 metadata: Dict,
                 config: EnterpriseConfig,
                 explainer: Optional[ModelExplainer] = None):
        
        self.model = model
        self.metadata = metadata
        self.config = config
        self.explainer = explainer
        self.artifact_id = self._generate_artifact_id()
        self.model_hash = self._compute_hash()
    
    def _generate_artifact_id(self) -> str:
        """Gera ID único para o artefato."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        return f"credit_model_v{self.config.ml_config.model_version}_{timestamp}"
    
    def _compute_hash(self) -> str:
        """Calcula hash SHA-256 do modelo."""
        model_bytes = joblib.dumps(self.model)
        return hashlib.sha256(model_bytes).hexdigest()
    
    def save(self, base_path: Optional[Path] = None):
        """Salva artefato completo."""
        
        if base_path is None:
            base_path = self.config.ml_config.output_dir
        
        artifact_dir = base_path / self.artifact_id
        artifact_dir.mkdir(parents=True, exist_ok=True)
        
        logger.info(f"Salvando artefato em: {artifact_dir}")
        
        # 1. Salva modelo
        model_path = artifact_dir / 'model.joblib'
        joblib.dump(self.model, model_path, compress=3)
        logger.info(f"   ✅ Modelo salvo: {model_path}")
        
        # 2. Salva metadados enriquecidos
        enriched_metadata = self._enrich_metadata()
        metadata_path = artifact_dir / 'metadata.json'
        with open(metadata_path, 'w') as f:
            json.dump(enriched_metadata, f, indent=4, default=str)
        logger.info(f"   ✅ Metadados salvos: {metadata_path}")
        
        # 3. Salva configuração
        config_path = artifact_dir / 'config.json'
        self.config.save(config_path)
        logger.info(f"   ✅ Configuração salva: {config_path}")
        
        # 4. Salva explainer (se disponível)
        if self.explainer is not None:
            explainer_path = artifact_dir / 'explainer.joblib'
            joblib.dump(self.explainer, explainer_path)
            logger.info(f"   ✅ Explainer salvo: {explainer_path}")
        
        # 5. Gera Model Card (documentação)
        if self.config.ml_config.generate_model_card:
            self._generate_model_card(artifact_dir)
        
        # 6. Cria arquivo de versão
        version_info = {
            'artifact_id': self.artifact_id,
            'model_hash': self.model_hash,
            'created_at': datetime.now().isoformat(),
            'python_version': sys.version,
            'dependencies': self._get_dependencies()
        }
        
        version_path = artifact_dir / 'version.json'
        with open(version_path, 'w') as f:
            json.dump(version_info, f, indent=4)
        logger.info(f"   ✅ Versão salva: {version_path}")
        
        logger.info(f"✅ Artefato completo salvo em: {artifact_dir}")
        
        return artifact_dir
    
    def _enrich_metadata(self) -> Dict:
        """Enriquece metadados com informações adicionais."""
        
        enriched = self.metadata.copy()
        enriched.update({
            'artifact_id': self.artifact_id,
            'model_hash': self.model_hash,
            'created_at': datetime.now().isoformat(),
            'model_version': self.config.ml_config.model_version,
            'framework_versions': {
                'xgboost': xgb.__version__,
                'lightgbm': lgb.__version__,
                'sklearn': __import__('sklearn').__version__,
                'pandas': pd.__version__,
                'numpy': np.__version__
            }
        })
        
        return enriched
    
    def _get_dependencies(self) -> Dict[str, str]:
        """Lista dependências principais."""
        
        return {
            'xgboost': xgb.__version__,
            'lightgbm': lgb.__version__,
            'scikit-learn': __import__('sklearn').__version__,
            'pandas': pd.__version__,
            'numpy': np.__version__,
            'shap': shap.__version__,
            'optuna': optuna.__version__
        }
    
    def _generate_model_card(self, artifact_dir: Path):
        """Gera Model Card (documentação do modelo)."""
        
        model_card = f"""
# Model Card: {self.artifact_id}

## Informações do Modelo

- **Nome**: Credit Scoring Model
- **Versão**: {self.config.ml_config.model_version}
- **Tipo**: {self.metadata.get('model_type', 'Ensemble')}
- **Data de Criação**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Hash do Modelo**: {self.model_hash}

## Objetivo

Modelo de credit scoring para predição de inadimplência em admissão de crédito.

## Performance

### Métricas de Treino
{self._format_metrics(self.metadata.get('train_metrics', {}))}

### Métricas de Validação
{self._format_metrics(self.metadata.get('validation_metrics', {}))}

### Métricas de Teste
{self._format_metrics(self.metadata.get('test_metrics', {}))}

### Métricas Out-of-Time (OOT)
{self._format_metrics(self.metadata.get('oot_metrics', {}))}

## Cutoff Ótimo

- **Threshold**: {self.metadata.get('optimal_cutoff', 0.5):.4f}
- **Estratégia**: {self.metadata.get('cutoff_strategy', 'N/A')}

## Features

### Quantidade de Features
- **Total**: {self.metadata.get('n_features', 'N/A')}
- **Numéricas**: {self.metadata.get('n_numeric_features', 'N/A')}
- **Categóricas**: {self.metadata.get('n_categorical_features', 'N/A')}

### Top 10 Features Mais Importantes
{self._format_feature_importance()}

## Dados de Treino

- **Total de Registros**: {self.metadata.get('n_samples_train', 'N/A')}
- **Período**: {self.metadata.get('train_period', 'N/A')}
- **Balanceamento**: {self.metadata.get('class_balance', 'N/A')}

## Validações

- **Data Leakage**: {self.metadata.get('leakage_check', 'PASSED')}
- **PSI (Train vs OOT)**: {self.metadata.get('psi_train_oot', 'N/A')}
- **Status de Drift**: {self.metadata.get('drift_status', 'OK')}

## Compliance

- **LGPD**: Compliant
- **BACEN Res. 4.557/2017**: Compliant
- **Audit Trail**: Enabled

## Uso
```python
import joblib
model = joblib.load('model.joblib')
predictions = model.predict_proba(X_new)[:, 1]
```

## Contato

Para dúvidas sobre este modelo, contate a equipe de Data Science.

---
*Gerado automaticamente em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""
        
        card_path = artifact_dir / 'MODEL_CARD.md'
        with open(card_path, 'w') as f:
            f.write(model_card)
        
        logger.info(f"   ✅ Model Card gerado: {card_path}")
    
    def _format_metrics(self, metrics: Dict) -> str:
        """Formata métricas para Model Card."""
        
        if not metrics:
            return "N/A"
        
        lines = []
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                if key in ['auc', 'gini', 'ks_statistic', 'precision', 'recall', 'f1_score']:
                    lines.append(f"- **{key.upper()}**: {value:.4f}")
                elif key in ['approval_rate', 'bad_rate']:
                    lines.append(f"- **{key}**: {value:.2%}")
                else:
                    lines.append(f"- **{key}**: {value}")
        
        return '\n'.join(lines) if lines else "N/A"
    
    def _format_feature_importance(self) -> str:
        """Formata importância de features para Model Card."""
        
        feature_importance = self.metadata.get('feature_importance', [])
        
        if not feature_importance:
            return "N/A"
        
        lines = []
        for i, feat in enumerate(feature_importance[:10], 1):
            if isinstance(feat, dict):
                lines.append(f"{i}. **{feat.get('feature', 'N/A')}**: {feat.get('importance', 0):.4f}")
            else:
                lines.append(f"{i}. {feat}")
        
        return '\n'.join(lines)
    
    @classmethod
    def load(cls, artifact_dir: Path) -> 'ModelArtifact':
        """Carrega artefato salvo."""
        
        logger.info(f"Carregando artefato de: {artifact_dir}")
        
        # Carrega modelo
        model_path = artifact_dir / 'model.joblib'
        model = joblib.load(model_path)
        
        # Carrega metadados
        metadata_path = artifact_dir / 'metadata.json'
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        # Carrega configuração
        config_path = artifact_dir / 'config.json'
        with open(config_path, 'r') as f:
            config_dict = json.load(f)
        
        # Reconstrói config (simplificado)
        config = EnterpriseConfig()
        
        # Carrega explainer se existir
        explainer_path = artifact_dir / 'explainer.joblib'
        explainer = None
        if explainer_path.exists():
            explainer = joblib.load(explainer_path)
        
        logger.info("✅ Artefato carregado com sucesso")
        
        return cls(model, metadata, config, explainer)
    
    def validate(self, X_sample: pd.DataFrame) -> bool:
        """Valida integridade do modelo."""
        
        try:
            # Testa predição
            predictions = self.model.predict_proba(X_sample)
            
            # Validações
            assert predictions.shape[1] == 2, "Modelo deve ter 2 classes"
            assert np.all((predictions >= 0) & (predictions <= 1)), "Probabilidades devem estar em [0, 1]"
            assert not np.any(np.isnan(predictions)), "Predições não podem conter NaN"
            
            # Verifica hash
            current_hash = self._compute_hash()
            if current_hash != self.model_hash:
                logger.warning("⚠️  Hash do modelo mudou! Possível corrupção.")
                return False
            
            logger.info("✅ Validação do modelo: PASSOU")
            return True
            
        except Exception as e:
            logger.error(f"❌ Validação falhou: {e}")
            return False

# ==============================================================================
# 13. PIPELINE PRINCIPAL DE TREINO
# ==============================================================================

# ==============================================================================
# CORREÇÃO FINAL: Adicionar remoção de colunas no _load_data()
# ==============================================================================

class EnterpriseCreditScoringPipeline:
    """Pipeline completo enterprise para Credit Scoring."""
    
    def __init__(self, config: Optional[EnterpriseConfig] = None):
        self.config = config or EnterpriseConfig()
        self.validator = DataValidator(self.config)
        self.splitter = TemporalDataSplitter(self.config.ml_config)
        self.optimizer = BayesianOptimizer(self.config)
        self.cutoff_optimizer = CutoffOptimizer(self.config.business_metrics)
        
        self.preprocessing_pipeline = None
        self.model = None
        self.model_artifact = None
        self.explainer = None
        
        self.results = {
            'train_metrics': {},
            'validation_metrics': {},
            'test_metrics': {},
            'oot_metrics': {},
            'feature_importance': [],
            'optimization_history': None
        }
    
    def run(self):
        """Executa pipeline completo."""
        
        logger.info("\n" + "="*70)
        logger.info("🚀 INICIANDO PIPELINE ENTERPRISE DE CREDIT SCORING")
        logger.info("="*70 + "\n")
        
        try:
            # 1. Carregamento e LIMPEZA de Dados
            logger.info("ETAPA 1/10: Carregamento e Limpeza de Dados")
            logger.info("-" * 70)
            df = self._load_and_clean_data()  # ← MODIFICADO
            
            # 2. Validação (agora sem colunas de leakage)
            logger.info("\nETAPA 2/10: Validação de Qualidade")
            logger.info("-" * 70)
            is_valid, validation_results = self.validator.validate_all(df)
            
            if not is_valid:
                raise ValueError("Dados falharam na validação. Verifique os logs.")
            
            # 3. Preparação de Dados
            logger.info("\nETAPA 3/10: Preparação de Dados")
            logger.info("-" * 70)
            X, y = self._prepare_data(df)
            
            # ... resto do código permanece igual
            
        except Exception as e:
            logger.error(f"\n❌ ERRO FATAL NO PIPELINE: {e}", exc_info=True)
            raise
    
    def _load_and_clean_data(self) -> pd.DataFrame:
        """Carrega dados E REMOVE colunas de leakage."""
        
        data_path = self.config.ml_config.data_path
        sheet_name = self.config.ml_config.sheet_name
        
        logger.info(f"Carregando dados de: {data_path}")
        df = pd.read_excel(data_path, sheet_name=sheet_name)
        logger.info(f"✅ Dados carregados: {df.shape[0]} registros, {df.shape[1]} colunas")
        
        # REMOVE COLUNAS DE LEAKAGE IMEDIATAMENTE
        cols_to_remove = [
            'saldo_vencido', 
            'quantidade_parcelas_vencidas',
            'primeiro_vencimento_em_atraso',
            'dias_em_atraso',
            'data_quitacao',
            'taxa_parcelas_vencidas',
            'recebido',
            'produtor',
            'lancamento',
            'pedido_id'
        ]
        
        cols_found = [col for col in cols_to_remove if col in df.columns]
        
        if cols_found:
            logger.info(f"🧹 Removendo {len(cols_found)} colunas de leakage/desnecessárias:")
            for col in cols_found:
                logger.info(f"   - {col}")
            df = df.drop(columns=cols_found)
            logger.info(f"✅ Dados limpos: {df.shape[1]} colunas restantes")
        
        return df
    
    def _prepare_data(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
        """Prepara dados para modelagem."""
        
        # Target
        target_col = self.config.ml_config.target_variable
        
        if target_col not in df.columns:
            raise ValueError(f"Coluna target '{target_col}' não encontrada")
        
        # Binariza target
        y = df[target_col].apply(lambda x: 1 if str(x).lower() in ['inadimplente', '1', 'true'] else 0)
        X = df.drop(columns=[target_col])
        
        logger.info(f"✅ Dados preparados:")
        logger.info(f"   Features: {X.shape[1]}")
        logger.info(f"   Target distribution:")
        logger.info(f"      Bom Pagador (0): {(y==0).sum()} ({(y==0).mean()*100:.2f}%)")
        logger.info(f"      Inadimplente (1): {(y==1).sum()} ({(y==1).mean()*100:.2f}%)")
        
        return X, y
    
    def _evaluate_all_sets(self, X_train, y_train, X_val, y_val, X_test, y_test, X_oot, y_oot):
        """Avalia modelo em todos os conjuntos."""
        
        sets = {
            'train': (X_train, y_train),
            'validation': (X_val, y_val),
            'test': (X_test, y_test),
            'oot': (X_oot, y_oot)
        }
        
        for set_name, (X, y) in sets.items():
            logger.info(f"\n  Avaliando {set_name.upper()}...")
            
            y_prob = self.model.predict_proba(X)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)
            
            metrics = CreditScoringMetrics.calculate_all_metrics(
                y.values, y_prob, y_pred
            )
            
            business_value = CreditScoringMetrics.calculate_business_value(
                metrics,
                self.config.business_metrics
            )
            
            metrics.update(business_value)
            
            self.results[f'{set_name}_metrics'] = metrics
            
            # Log principais métricas
            logger.info(f"    AUC: {metrics['auc']:.4f}")
            logger.info(f"    Gini: {metrics['gini']:.4f}")
            logger.info(f"    KS: {metrics['ks_statistic']:.4f}")
            logger.info(f"    Bad Rate: {metrics['bad_rate']:.2%}")
            logger.info(f"    Net Profit: R$ {metrics['net_profit']:,.2f}")
    
    def _save_model_artifact(self):
        """Salva artefato completo do modelo."""
        
        metadata = {
            'model_type': 'Calibrated Ensemble',
            'train_metrics': self.results['train_metrics'],
            'validation_metrics': self.results['validation_metrics'],
            'test_metrics': self.results['test_metrics'],
            'oot_metrics': self.results['oot_metrics'],
            'optimal_cutoff': self.results['optimal_cutoff'],
            'cutoff_strategy': self.results['cutoff_strategy'],
            'feature_importance': self.results['feature_importance'][:20],
            'n_features': len(self.results.get('feature_importance', [])),
            'leakage_check': 'PASSED'
        }
        
        self.model_artifact = ModelArtifact(
            model=self.model,
            metadata=metadata,
            config=self.config,
            explainer=self.explainer
        )
        
        artifact_dir = self.model_artifact.save()
        
        logger.info(f"\n✅ Artefato completo salvo em: {artifact_dir}")
    
    def _print_final_summary(self):
        """Imprime sumário final."""
        
        logger.info("\n" + "="*70)
        logger.info("📊 SUMÁRIO FINAL DE PERFORMANCE")
        logger.info("="*70)
        
        for set_name in ['train', 'validation', 'test', 'oot']:
            metrics = self.results.get(f'{set_name}_metrics', {})
            
            if metrics:
                logger.info(f"\n{set_name.upper()}:")
                logger.info(f"  AUC: {metrics.get('auc', 0):.4f}")
                logger.info(f"  Gini: {metrics.get('gini', 0):.4f}")
                logger.info(f"  KS: {metrics.get('ks_statistic', 0):.4f}")
                logger.info(f"  Approval Rate: {metrics.get('approval_rate', 0):.2%}")
                logger.info(f"  Bad Rate: {metrics.get('bad_rate', 0):.2%}")
                logger.info(f"  Net Profit: R$ {metrics.get('net_profit', 0):,.2f}")
        
        logger.info(f"\nCUTOFF ÓTIMO: {self.results.get('optimal_cutoff', 0):.4f}")
        logger.info("="*70)

# ==============================================================================
# 14. PONTO DE ENTRADA
# ==============================================================================

def main():
    """Função principal."""
    
    try:
        # Cria configuração
        config = EnterpriseConfig()
        
        # Customiza configuração se necessário
        config.ml_config.n_trials_optuna = 50  # Reduz para teste mais rápido
        
        # Cria e executa pipeline
        pipeline = EnterpriseCreditScoringPipeline(config)
        pipeline.run()
        
    except FileNotFoundError as e:
        logger.error(f"❌ Arquivo de dados não encontrado: {e}")
        logger.error("   Verifique o caminho em EnterpriseConfig.ml_config.data_path")
    
    except Exception as e:
        logger.error(f"❌ Erro fatal: {e}", exc_info=True)
        sys.exit(1)

if __name__ == '__main__':
    main()

# ==============================================================================
# FIM DO CÓDIGO ENTERPRISE 10/10
# ==============================================================================